In [ ]:
# ~~~ Imports ~~~
import subprocess
import os
import hydra
from aiflows.utils.general_helpers import read_yaml_file
from aiflows.backends.api_info import ApiInfo
from aiflows.utils.general_helpers import find_replace_in_dict
from aiflows.flow_launchers import FlowLauncher
from aiflows.backends.api_info import ApiInfo
from aiflows import flow_verse
from aiflows import logging

logging.set_verbosity_info()

### Defining Functions Which Reproduce The Steps Of The Quick Start Tutorial (no modifications required)

In [ ]:
# ~~~ Convenience function that install the dependencies for the Flows in this notebook ~~~
# Normally, you would install the dependencies in your terminal, as necessary. We do it here for you convenience

DEPENDENCIES_DICT = {
    "ChatFlowModule": "aiflows/ChatFlowModule",
    "ReAct": "aiflows/ControllerExecutorFlowModule",
    "ChatInteractiveFlowModule": "aiflows/ChatInteractiveFlowModule",
    "ChatWithDemonstrationsFlowModule": "aiflows/ChatWithDemonstrationsFlowModule",
    "AutoGPTFlowModule": "aiflows/AutoGPTFlowModule",
    "VisionFlowModule": "aiflows/VisionFlowModule",
}

def download_all_flow_dependencies(flow):
    """Download all dependencies for a flow from FlowVerse and pip requirements. 
    
    Note that this is a convenience function for this tutorial and is not meant to work doesn't work for all Flows on FlowVerse.
    
    :param flow: The name of the Flow to download dependencies for
    :type flow: str
    """
    # ~~~ Load Flow dependecies from FlowVerse ~~~
    dependencies = [
        {"url": DEPENDENCIES_DICT[flow], "revision": "main"},
    ]
    flow_verse.sync_dependencies(dependencies)
    
    # path to pip requirements file
    path_to_requirements = os.path.join("flow_modules" ,DEPENDENCIES_DICT[flow], "pip_requirements.txt")

    # download dependencies
    subprocess.check_call(["pip", "install", "-r", path_to_requirements], stdout=subprocess.DEVNULL)

# ~~~ Load Flow dependecies from FlowVerse and pip requirments ~~~
for key in DEPENDENCIES_DICT.keys():
    download_all_flow_dependencies(key)

In [ ]:
def run_flow(flow, data, api_information):
    """Loads the Flow config, instantiates the Flow and runs inference.
    
    :param flow: The name of the flow to run
    :type flow: str
    :param data: The input data to the flow
    :type data: dict
    :param api_information: The api information for the flow
    :type api_information: ApiInfo
    """
    
    flow_url = DEPENDENCIES_DICT[flow]

    # ~~~ Load Flow config ~~~
    path_to_config = os.path.join( "flow_modules" ,flow_url, "demo.yaml")
    cfg = read_yaml_file(path_to_config)

    # set the API information
    cfg = find_replace_in_dict(cfg, key_to_find="api_infos", new_value= api_information , current_path="")
  
    # ~~~ Instantiate the Flow ~~~
    flow_with_interfaces = {
        "flow": hydra.utils.instantiate(cfg["flow"], _recursive_=False, _convert_="partial"),
        "input_interface": None,
        "output_interface": None
    }
    
    # ~~~ Run inference ~~~
    path_to_output_file = None
    # path_to_output_file = "output.jsonl"  # Uncomment this line to save the output to disk

    _, outputs = FlowLauncher.launch(
        flow_with_interfaces=flow_with_interfaces, data=data, path_to_output_file=path_to_output_file
    )

    # ~~~ Print the output ~~~
    flow_output_data = outputs[0]
    print(flow_output_data)
    

### Setting you API Information

Set your API information here (copy-paste it)

In [ ]:

#OpenAI backend
api_key = "" # copy paste your api key here
api_information = [ApiInfo(backend_used="openai", api_key=api_key)]

# Azure backend
# api_key = "" # copy paste your api key here
# api_base = "" # copy paste your api base here
# api_version = "" #copypase your api base here
# api_information = ApiInfo(backend_used = "azure",
#                           api_base =api_base,
#                           api_key = api_version,
#                           api_version =  api_version )

### Pick a Flow, some data and run it!

Set the `Flow` and `data` values, then run the cell! Examples for each flow from the FlowVerse are provided in the cell bellow. Uncomment the flow and data you wish to run. Here is the list of flows available for you to run:

* [ChatFlowModule](https://huggingface.co/aiflows/ChatFlowModule)

* [ReAct](https://huggingface.co/aiflows/ControllerExecutorFlowModule)

* [AutoGPTFlowModule](https://huggingface.co/aiflows/AutoGPTFlowModule)

* [ChatInteractiveFlowModule](https://huggingface.co/aiflows/ChatInteractiveFlowModule)

* [ChatWithDemonstrationsFlowModule](https://huggingface.co/aiflows/ChatWithDemonstrationsFlowModule)

* [VisionFlowModule](https://huggingface.co/aiflows/VisionFlowModule)


In [ ]:
# Example for a simple chatbot
# [ChatFlowModule](https://huggingface.co/aiflows/ChatFlowModule)

flow = "ChatFlowModule"  
data = {"id": 0, "question": "Who was the NBA champion in 2023?"} 
run_flow(flow, data, api_information)

In [ ]:
# Example for ReAct (the simplest instance of a ControllerExecutorFlowModule)
flow = "ReAct"
data = {"id": 0, "goal": "Who was the NBA champion in 2023?"}
run_flow(flow, data, api_information)

In [ ]:
# Example for AutoGPTFlowModule
flow = "AutoGPTFlowModule"
data = {"id": 0, "goal": "Who was the NBA champion in 2023?"}
run_flow(flow, data, api_information)

In [ ]:
# Example for ChatInteractiveFlowModule
flow = "ChatInteractiveFlowModule"
data = {"id": 0} # you will be able to ask questions once it is running
run_flow(flow, data, api_information)

In [ ]:
# Example for ChatWithDemonstrationsFlowModule
flow = "ChatWithDemonstrationsFlowModule"
data = {"id": 0, "question": "What's the capital of France?"}  # This can be a list of samples
run_flow(flow, data, api_information)

In [ ]:
# Example for VisionFlowModule
flow = "VisionFlowModule"
url_image = {"type": "url",
            "image": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"}
data = {"id": 0, "query": "What’s in this image?", "data": {"images": [url_image]}}  # This can be a list of samples
run_flow(flow, data, api_information)